## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kozhva,RU,65.1073,57.0459,44.47,73,73,14.56,broken clouds
1,1,Hobart,AU,-42.8794,147.3294,54.27,84,20,10.36,few clouds
2,2,Henties Bay,NaN,-22.1160,14.2845,58.87,85,0,8.03,clear sky
3,3,Albany,US,42.6001,-73.9662,74.73,56,100,3.00,overcast clouds
4,4,Cherskiy,RU,68.7500,161.3000,36.37,65,99,13.78,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Henties Bay,NaN,-22.1160,14.2845,58.87,85,0,8.03,clear sky
3,3,Albany,US,42.6001,-73.9662,74.73,56,100,3.00,overcast clouds
7,7,Katsuura,JP,35.1333,140.3000,68.02,88,100,22.17,light rain
11,11,Avarua,CK,-21.2078,-159.7750,80.65,78,40,3.44,scattered clouds
13,13,East London,ZA,-33.0153,27.9116,62.13,80,100,5.57,overcast clouds
14,14,Esperance,AU,-33.8667,121.9000,56.62,82,2,5.19,clear sky
16,16,Port Alfred,ZA,-33.5906,26.8910,57.18,94,39,3.00,scattered clouds
17,17,Rehoboth,NaN,-23.3167,17.0833,61.97,22,0,5.28,clear sky
22,22,Cape Town,ZA,-33.9258,18.4232,60.91,85,3,7.00,clear sky
23,23,Nantucket,US,41.2835,-70.0995,63.14,81,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                464
City                   464
Country                455
Lat                    464
Lng                    464
Max Temp               464
Humidity               464
Cloudiness             464
Wind Speed             464
Current Description    464
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                455
City                   455
Country                455
Lat                    455
Lng                    455
Max Temp               455
Humidity               455
Cloudiness             455
Wind Speed             455
Current Description    455
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Albany,US,74.73,overcast clouds,42.6001,-73.9662,
7,Katsuura,JP,68.02,light rain,35.1333,140.3000,
11,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
13,East London,ZA,62.13,overcast clouds,-33.0153,27.9116,
14,Esperance,AU,56.62,clear sky,-33.8667,121.9000,
16,Port Alfred,ZA,57.18,scattered clouds,-33.5906,26.8910,
22,Cape Town,ZA,60.91,clear sky,-33.9258,18.4232,
23,Nantucket,US,63.14,clear sky,41.2835,-70.0995,
24,Manokwari,ID,77.65,overcast clouds,-0.8667,134.0833,
26,Rikitea,PF,77.29,clear sky,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.count()

City                   455
Country                455
Max Temp               455
Current Description    455
Lat                    455
Lng                    455
Hotel Name             455
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.nan).dropna()
clean_hotel_df.count()

City                   419
Country                419
Max Temp               419
Current Description    419
Lat                    419
Lng                    419
Hotel Name             419
dtype: int64

In [11]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Katsuura,JP,68.02,light rain,35.1333,140.3000,Katsuura Hotel Mikazuki
11,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
13,East London,ZA,62.13,overcast clouds,-33.0153,27.9116,Tu Casa
14,Esperance,AU,56.62,clear sky,-33.8667,121.9000,Hospitality Esperance
16,Port Alfred,ZA,57.18,scattered clouds,-33.5906,26.8910,The Halyards Hotel
22,Cape Town,ZA,60.91,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
23,Nantucket,US,63.14,clear sky,41.2835,-70.0995,Jared Coffin House
24,Manokwari,ID,77.65,overcast clouds,-0.8667,134.0833,Swiss Belhotel Manokwari
26,Rikitea,PF,77.29,clear sky,-23.1203,-134.9692,People ThankYou
28,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))